In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Code to webscape the table data from wikipedia page

In [26]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
df = df[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# Removal of Not Assigned values from column Borough

In [30]:
df = df[df['Borough'] != 'Not assigned']
df['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
East Toronto         5
York                 5
Mississauga          1
Name: Borough, dtype: int64

# Neighbourhood with 0 Not Assigned values

In [34]:
df[df.Neighborhood == 'Not assigned']

,Postal Code,Borough,Neighborhood


# Shaoe of dataframe

In [35]:
df.shape

(103, 3)